In [ ]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
world.plot(cmap='Set3', figsize=(10, 6))

In [ ]:
# reading the JSON data using json.load()
file = './data/GoogleLocation2018.json'
with open(file) as train_file:
    data_json = json.load(train_file)
    
df = pd.DataFrame({'raw': data_json['locations']})

In [ ]:
# Extract the base data
df['timeMs'] = df.apply(lambda row: int(row.raw['timestampMs']), axis=1)
df['lat'] = df.apply(lambda row: row.raw['latitudeE7'] / 10_000_000, axis=1)
df['lon'] = df.apply(lambda row: row.raw['longitudeE7'] / 10_000_000, axis=1)
df['acc'] = df.apply(lambda row: row.raw['accuracy'], axis=1)

In [ ]:
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
gdf = GeoDataFrame(df, geometry=geometry)

In [ ]:
gdf.plot(ax=world.plot(facecolor='lightgray', edgecolor='gray', figsize=(10, 6)),
             marker='o', color='red', markersize=15);

bounds = gdf.geometry.bounds

plt.xlim([bounds.minx.min()-5, bounds.maxx.max()+5])
plt.ylim([bounds.miny.min()-5, bounds.maxy.max()+5]);

In [ ]:
gdf.plot(ax=world.plot(facecolor='lightgray', edgecolor='gray', figsize=(10, 6)),
             marker='o', color='red', markersize=15);

bounds = gdf[gdf.long < 20].geometry.bounds

plt.xlim([bounds.minx.min()-2, bounds.maxx.max()+2])
plt.ylim([bounds.miny.min()-2, bounds.maxy.max()+2]);

In [ ]:
gdf.plot(ax=world.plot(facecolor='lightgray', edgecolor='gray', figsize=(10, 6)),
             marker='o', color='red', markersize=15);

bounds = gdf[gdf.long > 80].geometry.bounds

plt.xlim([bounds.minx.min()-2, bounds.maxx.max()+2])
plt.ylim([bounds.miny.min()-2, bounds.maxy.max()+2]);

In [ ]:
gdf.plot(ax=world.plot(facecolor='lightgray', edgecolor='gray', figsize=(10, 6)),
             marker='o', color='red', markersize=15);

bounds = gdf[(gdf.long > 30) & (gdf.long < 45)].geometry.bounds

plt.xlim([bounds.minx.min()-2, bounds.maxx.max()+2])
plt.ylim([bounds.miny.min()-2, bounds.maxy.max()+2]);